<a href="https://colab.research.google.com/github/abhishekrathoreiitb/maven-project/blob/master/Mt101_to_Pain001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
import re # Import regular expression module
import json # For pretty printing the parsed data
import uuid # For generating UETR
import traceback # For detailed error reporting

In [ ]:
# --- Namespaces ---
NS_SWINT = "urn:swift:snl:ns.SwInt"
NS_SWSEC = "urn:swift:snl:ns.SwSec"
NS_SW = "urn:swift:snl:ns.Sw"
NS_HEAD = "urn:iso:std:iso:20022:tech:xsd:head.001.001.02"
NS_PAIN = "urn:iso:std:iso:20022:tech:xsd:pain.001.001.09" # Target version

# Register namespaces for cleaner output
ET.register_namespace("SwInt", NS_SWINT)
ET.register_namespace("SwSec", NS_SWSEC)
ET.register_namespace("Sw", NS_SW)
# head and pain prefixes will be added locally via .set()

In [ ]:
# --- MT101 Parsing Function (Refined 50H/59 parsing for unstructured address) ---
def parse_swift_bic(field_value):
    """ Helper to extract BIC (8 or 11 chars) or None. """
    if not field_value: return None
    first_line = field_value.split('\n')[0].strip()
    if '/' in first_line: parts = first_line.split('/'); potential_bic = parts[-1].strip()
    else: potential_bic = first_line
    if (len(potential_bic) == 8 or len(potential_bic) == 11) and potential_bic[:6].isalnum() and potential_bic[6:].isalnum(): return potential_bic
    return None

In [ ]:
def parse_mt101(mt101_message):
    """
    Parses MT101 with refined 50H/59 logic for unstructured address lines.
    Args: mt101_message (str): The MT101 message.
    Returns: dict: Extracted data or None if parsing fails.
    """
    print("DEBUG: Entering parse_mt101")
    try:
        data = {}

        # --- Block Matching ---
        print("DEBUG: Searching for blocks...")
        block1_match = re.search(r'\{1:([^\}]+)\}', mt101_message)
        block2_match = re.search(r'\{2:([^\}]+)\}', mt101_message)
        block4_match = re.search(r'\{4:([\s\S]*?)\-?\}', mt101_message)
        if not block1_match: print("DEBUG: Error: Block 1 not found."); return None
        if not block2_match: print("DEBUG: Error: Block 2 not found."); return None
        if not block4_match: print("DEBUG: Error: Block 4 not found."); return None
        print("DEBUG: Blocks 1, 2, 4 found.")
        block1_content = block1_match.group(1); block2_content = block2_match.group(1); block4_content = block4_match.group(1).strip()

        # --- Block BIC Extraction ---
        print("DEBUG: Extracting block BICs...")
        data['sender_bic_block1'] = block1_content[3:11] if len(block1_content) >= 11 else 'N/A'
        data['receiver_bic_block2'] = 'N/A'
        if len(block2_content) >= 12:
             bic_part = block2_content[4:]
             if len(bic_part) >= 8:
                  potential_bic = bic_part[:8]
                  if len(bic_part) >= 11 and bic_part[8:11].isalnum() and bic_part[8:11] != 'XXX': potential_bic = bic_part[:11]
                  data['receiver_bic_block2'] = potential_bic
        print(f"DEBUG: sender_bic_block1 = {data.get('sender_bic_block1')}")
        print(f"DEBUG: receiver_bic_block2 = {data.get('receiver_bic_block2')}")

        # --- Parse Block 4 Tags ---
        print("DEBUG: Parsing Block 4 tags...")
        lines = block4_content.split('\n'); tag_values = {}; current_tag = None; current_option = None
        for line in lines:
            line = line.strip();
            if not line: continue
            if line.startswith(':'):
                parts = line.split(':', 2)
                if len(parts) == 3:
                    tag_part = parts[1]; value = parts[2].strip()
                    if len(tag_part) == 2 and tag_part.isdigit(): current_tag = tag_part; current_option = None
                    elif len(tag_part) == 3 and tag_part[:2].isdigit() and tag_part[2].isalpha(): current_tag = tag_part[:2]; current_option = tag_part[2]
                    else: current_tag = None; current_option = None; continue
                    full_tag_key = f"{current_tag}{current_option}" if current_option else current_tag
                    if full_tag_key not in tag_values: tag_values[full_tag_key] = value
                    else: tag_values[full_tag_key] += '\n' + value
                else: current_tag = None; current_option = None
            elif current_tag:
                 full_tag_key = f"{current_tag}{current_option}" if current_option else current_tag
                 if full_tag_key in tag_values: tag_values[full_tag_key] += '\n' + line
            else: pass
        data.update(tag_values)
        print(f"DEBUG: Parsed Block 4 tags: {list(tag_values.keys())}")

        # --- Specific Data Extraction (Refined 50H/59 for unstructured address) ---
        print("DEBUG: Extracting specific data points...")
        try:
            data['transaction_id'] = data.get('20', 'N/A'); data['end_to_end_id'] = data.get('21', 'N/A')
            payment_date_str = data.get('30'); data['payment_date'] = datetime.strptime(payment_date_str, '%y%m%d').strftime('%Y-%m-%d') if payment_date_str else 'N/A'
            currency_and_amount = data.get('32B', 'XXX0.00'); data['currency'] = currency_and_amount[:3] if len(currency_and_amount) >= 3 else 'XXX'
            amount_str = currency_and_amount[3:].replace(',', '') if len(currency_and_amount) > 3 else '0.00';
            if '.' not in amount_str and amount_str.isdigit(): amount_str += '.00';
            amount_str = re.sub(r"[^0-9.]", "", amount_str);
            try: float(amount_str); data['amount'] = amount_str
            except ValueError: data['amount'] = '0.00'; data['currency'] = 'XXX'; print(f"Warning: Invalid amount format in 32B '{currency_and_amount}'")

            # --- Debtor Info (50H / 50L) ---
            print("DEBUG: Parsing Debtor info (:50H/L:)...")
            debtor_field_val = data.get('50H') or data.get('50L')
            data['debtor_account_other_id'] = 'N/A'; data['debtor_name'] = 'N/A'; data['debtor_address_lines'] = [] # Initialize as list
            if debtor_field_val:
                lines = [line.strip() for line in debtor_field_val.split('\n') if line.strip()]
                line_offset = 0
                if lines and lines[0].startswith('/'):
                    data['debtor_account_other_id'] = lines[0][1:].strip()
                    line_offset = 1
                if len(lines) > line_offset: data['debtor_name'] = lines[line_offset]
                if len(lines) > line_offset + 1: # Collect remaining lines as address
                    data['debtor_address_lines'] = lines[line_offset + 1:]
            data['initiating_party_name'] = data['debtor_name']
            print(f"DEBUG: Debtor - Acct:'{data['debtor_account_other_id']}', Name:'{data['debtor_name']}', AddrLines:{data['debtor_address_lines']}")

            # --- Creditor Info (59 / 59F) ---
            print("DEBUG: Parsing Creditor info (:59/F:)...")
            creditor_field_val = data.get('59') or data.get('59F')
            data['creditor_account_other_id'] = 'N/A'; data['creditor_name'] = 'N/A'; data['creditor_address_lines'] = [] # Initialize as list
            if creditor_field_val:
                lines = [line.strip() for line in creditor_field_val.split('\n') if line.strip()]
                line_offset = 0
                if lines and lines[0].startswith('/'):
                    data['creditor_account_other_id'] = lines[0][1:].strip()
                    line_offset = 1
                if len(lines) > line_offset: data['creditor_name'] = lines[line_offset]
                if len(lines) > line_offset + 1: # Collect remaining lines as address
                    data['creditor_address_lines'] = lines[line_offset + 1:]
            print(f"DEBUG: Creditor - Acct:'{data['creditor_account_other_id']}', Name:'{data['creditor_name']}', AddrLines:{data['creditor_address_lines']}")


            # --- Other Fields ---
            data['debtor_agent_bic'] = parse_swift_bic(data.get('52A') or data.get('52D'))
            data['creditor_agent_bic'] = parse_swift_bic(data.get('57A') or data.get('57D'))
            data['remittance_information'] = data.get('70', 'N/A')
            # Store the raw 71A value for mapping later
            data['charge_bearer_raw_71A'] = data.get('71A', 'N/A') # Store raw value
            # Remove the old charge_bearer key if it exists from previous logic
            data.pop('charge_bearer', None)

            data['forwarding_agent_bic'] = data.get('receiver_bic_block2') # Default

            print("DEBUG: Finished specific data extraction.")

        except Exception as e:
            print(f"DEBUG: Error during specific data extraction: {e}")
            traceback.print_exc()
            return None

        # --- Final BIC Checks ---
        print("DEBUG: Performing final BIC checks...")
        if (data.get('debtor_agent_bic') is None and data.get('sender_bic_block1') == 'N/A'): print("DEBUG: Error: Missing effective Debtor Agent BIC."); return None
        if (data.get('creditor_agent_bic') is None and data.get('receiver_bic_block2') == 'N/A'): print("DEBUG: Error: Missing effective Creditor Agent BIC."); return None
        print("DEBUG: Final BIC checks passed.")

        print("DEBUG: parse_mt101 successful.")
        return data
    except Exception as e:
        print(f"DEBUG: Critical MT101 parsing error: {e}")
        traceback.print_exc()
        return None

In [ ]:
# --- XML Generation Function (Updated for Unstructured Address & ChrgBr Mapping) ---
def mt101_to_pain001(mt101_data):
    """
    Generates SWIFTNet XML including pain.001.001.09, using parsed data,
    unstructured addresses, and specific charge bearer mapping.
    Args: mt101_data (dict): Dictionary from parse_mt101.
    Returns: str: The complete XML string, or None if error.
    """
    try:
        # --- Data Validation & Resolution ---
        required_keys = ['sender_bic_block1', 'receiver_bic_block2', 'transaction_id',
                         'payment_date', 'currency', 'amount', 'debtor_name',
                         'debtor_account_other_id', 'creditor_name',
                         'creditor_account_other_id', 'end_to_end_id']
        missing_keys = [k for k in required_keys if mt101_data.get(k) in [None, 'N/A'] or str(mt101_data.get(k)).startswith('MISSING_') or str(mt101_data.get(k)).startswith('PLACEHOLDER_')]
        if missing_keys: print(f"Error generating XML: Missing essential data for keys: {missing_keys}"); return None
        dbtr_agt_bic_resolved = mt101_data.get('debtor_agent_bic') or mt101_data.get('sender_bic_block1')
        cdtr_agt_bic_resolved = mt101_data.get('creditor_agent_bic') or mt101_data.get('receiver_bic_block2')
        fwdg_agt_bic_resolved = mt101_data.get('forwarding_agent_bic') or mt101_data.get('receiver_bic_block2')
        if not dbtr_agt_bic_resolved or dbtr_agt_bic_resolved == 'N/A': print("Error generating XML: Missing effective Debtor Agent BIC."); return None
        if not cdtr_agt_bic_resolved or cdtr_agt_bic_resolved == 'N/A': print("Error generating XML: Missing effective Creditor Agent BIC."); return None
        if not fwdg_agt_bic_resolved or fwdg_agt_bic_resolved == 'N/A': print("Error generating XML: Missing effective Forwarding Agent BIC."); return None

        # --- Charge Bearer Mapping ---
        charge_bearer_71a = mt101_data.get('charge_bearer_raw_71A', 'N/A')
        chrg_br_pain_value = None # Default to omitting the tag
        if charge_bearer_71a == 'SHA': chrg_br_pain_value = 'SHAR'
        elif charge_bearer_71a == 'OUR': chrg_br_pain_value = 'DEBT'
        elif charge_bearer_71a == 'BEN': chrg_br_pain_value = 'CRED'
        print(f"DEBUG: Mapping 71A '{charge_bearer_71a}' to ChrgBr '{chrg_br_pain_value}'")


        # --- Create Root Element ---
        root = ET.Element(f"{{{NS_SWINT}}}ExchangeRequest")
        # --- SWIFTNet Envelope ---
        auth_ctx = ET.SubElement(root, f"{{{NS_SWSEC}}}AuthorisationContext"); ET.SubElement(auth_ctx, f"{{{NS_SWSEC}}}UserDN").text = "self-or-descendant"; ET.SubElement(auth_ctx, f"{{{NS_SWSEC}}}AuthorisationContext")
        ET.SubElement(root, f"{{{NS_SWINT}}}Themes")
        req_ctrl1 = ET.SubElement(root, f"{{{NS_SWINT}}}RequestControl"); ET.SubElement(req_ctrl1, f"{{{NS_SWINT}}}RequestCrypto").text = "FALSE"; ET.SubElement(req_ctrl1, f"{{{NS_SWINT}}}NRIndicator").text = "FALSE"
        delivery_ctrl = ET.SubElement(root, f"{{{NS_SWINT}}}DeliveryCtrl"); ET.SubElement(delivery_ctrl, f"{{{NS_SWINT}}}DeliveryMode").text = "SNF"; ET.SubElement(delivery_ctrl, f"{{{NS_SWINT}}}NotifyQueue").text = mt101_data.get("notify_queue", f"{fwdg_agt_bic_resolved}.finplusfut"); ET.SubElement(delivery_ctrl, f"{{{NS_SWINT}}}DeliveryNotif").text = "FALSE"; ET.SubElement(delivery_ctrl, f"{{{NS_SWINT}}}DeliveryNotificationViewSystemMessage").text = "TRUE"
        prod_list = ET.SubElement(root, f"{{{NS_SW}}}ProductList"); prod_info = ET.SubElement(prod_list, f"{{{NS_SW}}}ProductInfo"); ET.SubElement(prod_info, f"{{{NS_SW}}}VendorName").text = "SNMQBEBB"; ET.SubElement(prod_info, f"{{{NS_SW}}}ProductName").text = "AMM"; ET.SubElement(prod_info, f"{{{NS_SW}}}ProductVersion").text = "4.1.00"
        req_ctrl2 = ET.SubElement(root, f"{{{NS_SWINT}}}RequestControl"); ET.SubElement(req_ctrl2, f"{{{NS_SWINT}}}RequestSubType").text = mt101_data.get("swift_request_subtype", "swift.cbprplus.02")
        req_e2e_ctrl = ET.SubElement(root, f"{{{NS_SWINT}}}RequestE2ECtrl"); creation_time_utc = datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'); ET.SubElement(req_e2e_ctrl, f"{{{NS_SW}}}CreationTime").text = creation_time_utc
        req_header = ET.SubElement(root, f"{{{NS_SWINT}}}RequestHeader"); ET.SubElement(req_header, f"{{{NS_SWINT}}}Requestor").text = mt101_data.get("swift_requestor_dn", f"ou=xxx,o={mt101_data['sender_bic_block1'][:8].lower()},o=swift"); ET.SubElement(req_header, f"{{{NS_SWINT}}}Responder").text = mt101_data.get("swift_responder_dn", f"ou=xxx,o={fwdg_agt_bic_resolved[:8].lower()},o=swift"); ET.SubElement(req_header, f"{{{NS_SWINT}}}Service").text = mt101_data.get("swift_service", "swift.finplus!pf"); ET.SubElement(req_header, f"{{{NS_SWINT}}}RequestType").text = "pain.001.001.09"; ET.SubElement(req_header, f"{{{NS_SWINT}}}RequestRef").text = mt101_data.get("swift_request_ref", "MNANEW")
        # --- SwInt:RequestPayload ---
        req_payload = ET.SubElement(root, f"{{{NS_SWINT}}}RequestPayload")
        # --- head:AppHdr ---
        app_hdr = ET.Element(f"{{{NS_HEAD}}}AppHdr"); app_hdr.set(f"xmlns:head", NS_HEAD)
        fr_fiid = ET.SubElement(ET.SubElement(app_hdr, f"{{{NS_HEAD}}}Fr"), f"{{{NS_HEAD}}}FIId"); fr_fininstnid = ET.SubElement(fr_fiid, f"{{{NS_HEAD}}}FinInstnId"); ET.SubElement(fr_fininstnid, f"{{{NS_HEAD}}}BICFI").text = mt101_data['sender_bic_block1']
        to_fiid = ET.SubElement(ET.SubElement(app_hdr, f"{{{NS_HEAD}}}To"), f"{{{NS_HEAD}}}FIId"); to_fininstnid = ET.SubElement(to_fiid, f"{{{NS_HEAD}}}FinInstnId"); ET.SubElement(to_fininstnid, f"{{{NS_HEAD}}}BICFI").text = fwdg_agt_bic_resolved
        ET.SubElement(app_hdr, f"{{{NS_HEAD}}}BizMsgIdr").text = mt101_data['transaction_id']; ET.SubElement(app_hdr, f"{{{NS_HEAD}}}MsgDefIdr").text = "pain.001.001.09"; ET.SubElement(app_hdr, f"{{{NS_HEAD}}}BizSvc").text = mt101_data.get("swift_biz_svc", "swift.cbprplus.02")
        bah_credt = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%S+00:00'); ET.SubElement(app_hdr, f"{{{NS_HEAD}}}CreDt").text = bah_credt
        req_payload.append(app_hdr)
        # --- pain:Document ---
        pain_doc = ET.Element(f"{{{NS_PAIN}}}Document"); pain_doc.set(f"xmlns:pain", NS_PAIN)
        pain_cstmr_initn = ET.SubElement(pain_doc, f"{{{NS_PAIN}}}CstmrCdtTrfInitn")
        # pain:GrpHdr
        pain_grp_hdr = ET.SubElement(pain_cstmr_initn, f"{{{NS_PAIN}}}GrpHdr"); ET.SubElement(pain_grp_hdr, f"{{{NS_PAIN}}}MsgId").text = mt101_data['transaction_id']; ET.SubElement(pain_grp_hdr, f"{{{NS_PAIN}}}CreDtTm").text = bah_credt; ET.SubElement(pain_grp_hdr, f"{{{NS_PAIN}}}NbOfTxs").text = "1"
        pain_initg_pty = ET.SubElement(pain_grp_hdr, f"{{{NS_PAIN}}}InitgPty"); ET.SubElement(pain_initg_pty, f"{{{NS_PAIN}}}Nm").text = mt101_data['initiating_party_name']
        if fwdg_agt_bic_resolved: pain_fwdg_agt = ET.SubElement(pain_grp_hdr, f"{{{NS_PAIN}}}FwdgAgt"); pain_fwdg_agt_fi = ET.SubElement(pain_fwdg_agt, f"{{{NS_PAIN}}}FinInstnId"); ET.SubElement(pain_fwdg_agt_fi, f"{{{NS_PAIN}}}BICFI").text = fwdg_agt_bic_resolved
        # pain:PmtInf
        pain_pmt_inf = ET.SubElement(pain_cstmr_initn, f"{{{NS_PAIN}}}PmtInf"); ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}PmtInfId").text = mt101_data['transaction_id']; ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}PmtMtd").text = "TRF"; pain_reqd_exctn_dt = ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}ReqdExctnDt"); ET.SubElement(pain_reqd_exctn_dt, f"{{{NS_PAIN}}}Dt").text = mt101_data['payment_date']
        pain_dbtr = ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}Dbtr"); ET.SubElement(pain_dbtr, f"{{{NS_PAIN}}}Nm").text = mt101_data['debtor_name']
        # Add Debtor Address using AdrLine if available
        debtor_addr_lines = mt101_data.get('debtor_address_lines', [])
        if debtor_addr_lines:
             pain_dbtr_addr = ET.SubElement(pain_dbtr, f"{{{NS_PAIN}}}PstlAdr")
             for line in debtor_addr_lines:
                 ET.SubElement(pain_dbtr_addr, f"{{{NS_PAIN}}}AdrLine").text = line

        pain_dbtr_acct = ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}DbtrAcct"); pain_dbtr_acct_id = ET.SubElement(pain_dbtr_acct, f"{{{NS_PAIN}}}Id"); pain_dbtr_acct_othr = ET.SubElement(pain_dbtr_acct_id, f"{{{NS_PAIN}}}Othr"); ET.SubElement(pain_dbtr_acct_othr, f"{{{NS_PAIN}}}Id").text = mt101_data['debtor_account_other_id']; ET.SubElement(pain_dbtr_acct, f"{{{NS_PAIN}}}Ccy").text = mt101_data['currency']
        pain_dbtr_agt = ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}DbtrAgt"); pain_dbtr_agt_fi = ET.SubElement(pain_dbtr_agt, f"{{{NS_PAIN}}}FinInstnId"); ET.SubElement(pain_dbtr_agt_fi, f"{{{NS_PAIN}}}BICFI").text = dbtr_agt_bic_resolved
        # pain:CdtTrfTxInf
        pain_cdt_trf_tx_inf = ET.SubElement(pain_pmt_inf, f"{{{NS_PAIN}}}CdtTrfTxInf"); pain_pmt_id = ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}PmtId"); ET.SubElement(pain_pmt_id, f"{{{NS_PAIN}}}InstrId").text = mt101_data['transaction_id']; ET.SubElement(pain_pmt_id, f"{{{NS_PAIN}}}EndToEndId").text = mt101_data['end_to_end_id']; uet_ref = str(uuid.uuid4()); ET.SubElement(pain_pmt_id, f"{{{NS_PAIN}}}UETR").text = uet_ref
        pain_amt = ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}Amt"); ET.SubElement(pain_amt, f"{{{NS_PAIN}}}InstdAmt", Ccy=mt101_data['currency']).text = mt101_data['amount']

        # Add Charge Bearer using mapped value if available
        if chrg_br_pain_value:
            ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}ChrgBr").text = chrg_br_pain_value

        pain_cdtr_agt = ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}CdtrAgt"); pain_cdtr_agt_fi = ET.SubElement(pain_cdtr_agt, f"{{{NS_PAIN}}}FinInstnId"); ET.SubElement(pain_cdtr_agt_fi, f"{{{NS_PAIN}}}BICFI").text = cdtr_agt_bic_resolved
        pain_cdtr = ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}Cdtr"); ET.SubElement(pain_cdtr, f"{{{NS_PAIN}}}Nm").text = mt101_data['creditor_name']
        # Add Creditor Address using AdrLine if available
        creditor_addr_lines = mt101_data.get('creditor_address_lines', [])
        if creditor_addr_lines:
             pain_cdtr_addr = ET.SubElement(pain_cdtr, f"{{{NS_PAIN}}}PstlAdr")
             for line in creditor_addr_lines:
                  ET.SubElement(pain_cdtr_addr, f"{{{NS_PAIN}}}AdrLine").text = line

        pain_cdtr_acct = ET.SubElement(pain_cdt_trf_tx_inf, f"{{{NS_PAIN}}}CdtrAcct"); pain_cdtr_acct_id = ET.SubElement(pain_cdtr_acct, f"{{{NS_PAIN}}}Id"); pain_cdtr_acct_othr = ET.SubElement(pain_cdtr_acct_id, f"{{{NS_PAIN}}}Othr"); ET.SubElement(pain_cdtr_acct_othr, f"{{{NS_PAIN}}}Id").text = mt101_data['creditor_account_other_id']
        req_payload.append(pain_doc)

        # --- Final Formatting ---
        ET.indent(root, space="    ", level=0)
        xml_string = ET.tostring(root, encoding="unicode", method="xml")
        xml_declaration = '<?xml version="1.0" encoding="UTF-8"?>\n'
        return xml_declaration + xml_string

    except Exception as e:
        print(f"Error generating SWIFTNet XML: {e}")
        traceback.print_exc()
        return None

In [ ]:
# --- MT101 Message Input (User Provided) ---
mt101_message = """
{1:F01FNBCUS44A1230000000000}
{2:I101BOFAUS6SX123N2}
{4:
:20:091117-DSNY0001
:21:WDC091117RPCUS
:28D:1/1
:30:091117
:32B:USD377250,
:50H:/12345-67891
WALT DISNEY COMPANY
MOUSE STREET 1
LOS ANGELES CA
:52A:BKTRUS33XXX
:57A:PNBPUS3NXXX
:59:/26351-38947
RIVERS PAPER COMPANY
37498 STONE ROAD
SAN RAMON CA
:70:Invoice 123 Payment For Services Rendered
:71A:SHA
-}
"""

In [ ]:
# --- Conversion and Output ---
print("--- Starting MT101 Parsing ---")
parsed_mt101_data = parse_mt101(mt101_message)
print("--- Finished MT101 Parsing ---")

if parsed_mt101_data:
     # --- Enrichment (If needed, e.g., missing account IDs if not parsed) ---
     # If parsing correctly gets account IDs from 50H/59, these might not be needed
     # parsed_mt101_data['debtor_account_other_id'] = "12345-67891" # Example if needed
     # parsed_mt101_data['creditor_account_other_id'] = "26351-38947" # Example if needed
     # parsed_mt101_data['forwarding_agent_bic'] = "..." # If different from receiver

     print("\n--- Parsed & Enriched MT101 Data ---")
     filtered_data = {k: v for k, v in parsed_mt101_data.items() if not re.match(r'^\d{2}[A-Z]?$', k)}
     print(json.dumps(filtered_data, indent=2))

     print("\n--- Starting SWIFTNet XML Generation ---")
     swiftnet_xml_output = mt101_to_pain001(parsed_mt101_data) # Use the generation function

     if swiftnet_xml_output:
         print("\n--- Generated SWIFTNet XML Output ---")
         print(swiftnet_xml_output)
     else:
         print("\nXML generation failed.")
else:
    print("\nMT101 parsing failed.")

--- Starting MT101 Parsing ---
DEBUG: Entering parse_mt101
DEBUG: Searching for blocks...
DEBUG: Blocks 1, 2, 4 found.
DEBUG: Extracting block BICs...
DEBUG: sender_bic_block1 = FNBCUS44
DEBUG: receiver_bic_block2 = BOFAUS6SX12
DEBUG: Parsing Block 4 tags...
DEBUG: Parsed Block 4 tags: ['20', '21', '28D', '30', '32B', '50H', '52A', '57A', '59', '70', '71A']
DEBUG: Extracting specific data points...
DEBUG: Parsing Debtor info (:50H/L:)...
DEBUG: Debtor - Acct:'12345-67891', Name:'WALT DISNEY COMPANY', AddrLines:['MOUSE STREET 1', 'LOS ANGELES CA']
DEBUG: Parsing Creditor info (:59/F:)...
DEBUG: Creditor - Acct:'26351-38947', Name:'RIVERS PAPER COMPANY', AddrLines:['37498 STONE ROAD', 'SAN RAMON CA']
DEBUG: Finished specific data extraction.
DEBUG: Performing final BIC checks...
DEBUG: Final BIC checks passed.
DEBUG: parse_mt101 successful.
--- Finished MT101 Parsing ---

--- Parsed & Enriched MT101 Data ---
{
  "sender_bic_block1": "FNBCUS44",
  "receiver_bic_block2": "BOFAUS6SX12",
  "